In [35]:
import imaplib
import email
import time

from email.message import EmailMessage

In [2]:
mailbox1_credentials = ('source@gt-soft.eu', 'xxx', 'server.eu')

In [3]:
mailbox2_credentials = ('destination@gt-soft.eu', 'xxx#@!', 'server.pl')

In [4]:
mailbox1 = imaplib.IMAP4_SSL(mailbox1_credentials[2])
mailbox1.login(mailbox1_credentials[0], mailbox1_credentials[1])

('OK', [b'Logged in'])

In [5]:
mailbox2 = imaplib.IMAP4_SSL(mailbox2_credentials[2])
mailbox2.login(mailbox2_credentials[0], mailbox2_credentials[1])

('OK', [b'Logged in'])

In [6]:
mailbox1_folders = mailbox1.list()[1]
mailbox2_folders = mailbox2.list()[1]

In [7]:
def print_folder_structure(mailbox, prefix=''):
    _, folders = mailbox.list('"{0}"'.format(prefix))
    for folder in folders:
        folder_name = folder.decode().split(' "." ')[-1].strip('"')
        print(prefix + folder_name)
        if '\\HasChildren' in folder.decode():
            print_folder_structure(mailbox, prefix=prefix + folder_name + '.')


In [8]:
print_folder_structure(mailbox1)

Junk
Accounts
Accounts.Accounts.NVIDIA
Accounts.Accounts.Renault
Accounts.NVIDIA
Accounts.Renault
Trash
INBOX.spam
INBOX.Drafts
INBOX.Sent
INBOX.Trash
Drafts
Sent
INBOX
INBOX.INBOX.spam
INBOX.INBOX.Drafts
INBOX.INBOX.Sent
INBOX.INBOX.Trash


In [9]:
print_folder_structure(mailbox2)

Sent
Drafts
Trash
INBOX.Trash
INBOX.Sent
INBOX.Drafts
INBOX.spam
Accounts
Accounts.Accounts.Accounts


AttributeError: 'NoneType' object has no attribute 'decode'

In [ ]:
from email.header import decode_header

def clean_header_value(header_value):
    if isinstance(header_value, str):
        if '=?' in header_value:
            decoded_header = decode_header(header_value)
            return ''.join([str(t[0], t[1] or 'ascii') for t in decoded_header])
        else:
            # Remove newlines and tabs
            header_value = re.sub(r'[\n\t]', '', header_value)

            # Handle incomplete email addresses
            if '@' in header_value and '.' not in header_value:
                header_value += '.invalid'

            return header_value
    else:
        return header_value


In [15]:
def transfer_emails(src_mailbox, dest_mailbox, folder_name):
    print(f"Trying to select folder '{folder_name}'")
    try:
        src_mailbox.select(folder_name)
    except imaplib.IMAP4.error as e:
        print(f"Error selecting folder '{folder_name}': {e}")
        return

    print(f"Folder '{folder_name}' selected")
    typ, data = src_mailbox.search(None, 'ALL')
    for num in data[0].split():
        typ, msg_data = src_mailbox.fetch(num, '(RFC822)')
        msg = email.message_from_bytes(msg_data[0][1])
        dest_mailbox.append(folder_name, '', imaplib.Time2Internaldate(time.time()), msg.as_bytes())


In [30]:
def create_folder(mailbox, folder_name):
    status, response = mailbox.create(folder_name)
    if status == 'OK':
        print(f'Successfully created folder "{folder_name}" in the destination mailbox.')
        return True
    elif b'[ALREADYEXISTS]' in response[0]:
        print(f'Folder "{folder_name}" already exists in the destination mailbox.')
        return True
    else:
        print(f'Failed to create folder "{folder_name}" in the destination mailbox. Response: {response}')
        return False


In [31]:
def folder_exists(mailbox, folder_name):
    status, response = mailbox.select(folder_name, readonly=True)
    if status == 'OK':
        return True
    else:
        return False


In [32]:
def find_next_folder(mailbox, parent_folder=None):
    if parent_folder is not None:
        search_pattern = f"{parent_folder}.*"
    else:
        search_pattern = "*"

    typ, data = mailbox.list(directory=parent_folder, pattern=search_pattern)
    for item in data:
        if item:
            folder_name = item.decode().split(' ')[-1].strip('"')
            if folder_name not in processed_folders:
                return folder_name
    return None


In [36]:
processed_folders = set()

def process_folder(src_mailbox, dest_mailbox, folder, parent_folder=None):
    # Clean up the folder name
    full_folder_name = folder.decode().split(' ')[-1].strip('"')
    if parent_folder is not None:
        full_folder_name = f"{parent_folder}.{full_folder_name}"

    print(f"Processing folder '{full_folder_name}'")

    # Create the folder in the destination mailbox
    created = create_folder(dest_mailbox, full_folder_name)
    if not created:
        return

    # Transfer emails from source to destination mailbox
    transfer_emails(src_mailbox, dest_mailbox, full_folder_name)

    processed_folders.add(full_folder_name)

    # Process child folders
    child_folder = find_next_folder(src_mailbox, full_folder_name)
    if child_folder is not None:
        child_folder_name = child_folder.decode().split(' ')[-1].strip('"')
        process_folder(src_mailbox, dest_mailbox, child_folder_name, full_folder_name)


In [37]:
for folder in mailbox1_folders:
    process_folder(mailbox1, mailbox2, folder)

Processing folder 'Junk'
Folder "Junk" already exists in the destination mailbox.
Trying to select folder 'Junk'
Folder 'Junk' selected
Processing folder 'Accounts'
Folder "Accounts" already exists in the destination mailbox.
Trying to select folder 'Accounts'
Folder 'Accounts' selected
Processing folder 'Accounts.NVIDIA'
Folder "Accounts.NVIDIA" already exists in the destination mailbox.
Trying to select folder 'Accounts.NVIDIA'
Folder 'Accounts.NVIDIA' selected
Processing folder 'Accounts.Renault'
Folder "Accounts.Renault" already exists in the destination mailbox.
Trying to select folder 'Accounts.Renault'
Folder 'Accounts.Renault' selected
Processing folder 'Trash'
Folder "Trash" already exists in the destination mailbox.
Trying to select folder 'Trash'
Folder 'Trash' selected
Processing folder 'INBOX.spam'
Folder "INBOX.spam" already exists in the destination mailbox.
Trying to select folder 'INBOX.spam'
Folder 'INBOX.spam' selected
Processing folder 'INBOX.Drafts'
Folder "INBOX.D

In [38]:
# Clean up
mailbox1.logout()

('BYE', [b'Logging out'])

In [39]:
# Clean up
mailbox2.logout()

('BYE', [b'Logging out'])